# Observation / Action Specifiction (Clean Implementation)

TODO:
- [x] Observation Spec `replay_df`
   - [x] Champs
   - [x] Minions
   - [x] Turrets
   - [x] Monsters
   - [x] Missiles
- [x] Action Spec `replay_df`:
   - [x] Movement
   - [-] Recall (Code available)
   - [x] Spell
     - [x] Q
     - [x] W
     - [x] E (Get direction in digits)
     - [x] D (Flash)
     - [-] F (Ignore this for now)
   - [x] Auto Attack
- [ ] Normalisation and Embedding
   - [ ] Convert strings into embeddings
      - [x] Straight up encode all known string lists
      - [ ] Hash open-ended lists (spell name, etc.) and then encode using the hash list
   - [ ] Normalise numeric values (Min-Max, what type of norm to use here?)

## Populate Game Object Dataframes

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

db_replays_dir = "/Users/joe/Downloads/DB"
db_replays = os.listdir(db_replays_dir)

In [2]:
with open("list_of_champs.txt") as f:
    CHAMP_LIST = f.read().split("\n")

In [3]:
import joblib
scaler = MinMaxScaler(feature_range=(0, 1))

In [4]:
import sqlite3
import pandas as pd
import os
db_replay = os.path.join(db_replays_dir, db_replays[0])
con = sqlite3.connect(db_replay)

In [5]:
SUMMONER_NAMES = ["SummonerBarrier", "SummonerBoost", "SummonerDot", "SummonerExhaust", "SummonerFlash", "SummonerHaste", "SummonerHeal", "SummonerMana", "SummonerTeleport", "SummonerSmite"]
MINION_NAMES = ['SRU_ChaosMinionMelee', 'SRU_OrderMinionMelee',
       'SRU_ChaosMinionRanged', 'SRU_OrderMinionRanged',
       'SRU_Plant_Vision', 'SRU_OrderMinionSiege', 'SRU_ChaosMinionSiege',
       'TestCubeRender']

In [6]:
AUTO_ATTACK_TARGETS = ["CHAMPS", "TURRETS", "MINIONS", "MISSILES", "MONSTERS", "OTHER"]

In [7]:
GAME_OBJECT_LIST = ["champs", "turrets", "minions", "missiles", "monsters"]

In [8]:
MAX_OBJS = [10, 30, 30, 30, 30]

In [9]:
df_s = {
    obj:pd.read_sql(f"SELECT * FROM {obj};", con).drop(labels=["game_id"], axis=1) for obj in GAME_OBJECT_LIST}

## Dataframe Preprocessing

### Clean `missiles_df`

In [10]:
df_s["missiles"] = df_s["missiles"].drop(labels=["name", "src_idx", "dst_idx"], axis=1)

### Convert Champ Names into Index

In [11]:
df_s["champs"] = df_s["champs"][df_s["champs"]["name"].astype(bool)]

In [12]:
df_s["champs"]["name"].unique()

array(['Akali', 'Evelynn', 'Camille', 'Ezreal', 'Rakan', 'Gnar', 'Viego',
       'Ryze', 'Xayah', 'Thresh'], dtype=object)

In [13]:
df_s["champs"]["name"] = df_s["champs"]["name"].apply(lambda d: CHAMP_LIST.index(d))

In [14]:
df_s["champs"]["d_name"] = df_s["champs"]["d_name"].apply(lambda d: SUMMONER_NAMES.index(d))
df_s["champs"]["f_name"] = df_s["champs"]["f_name"].apply(lambda d: SUMMONER_NAMES.index(d))

### Convert Champ Spells into Index

In [92]:
# Full champ spells list which needs to be feature hashed
# q_name, w_name, e_name, r_name
df_s["champs"].q_name
df_s["champs"].w_name
df_s["champs"].e_name
df_s["champs"].r_name

q_name      q_cd    w_name  w_cd    e_name      e_cd  \
0           AkaliQ  0.000000    AkaliW   0.0    AkaliE  0.000000   
1         EvelynnQ  0.000000  EvelynnW   0.0  EvelynnE  0.000000   
2         CamilleQ  0.000000  CamilleW   0.0  CamilleE  0.000000   
3          EzrealQ  0.000000   EzrealW   0.0   EzrealE  0.000000   
4           RakanQ  0.000000    RakanW   0.0    RakanE  0.000000   
...            ...       ...       ...   ...       ...       ...   
5735         GnarQ  3.875977     GnarW   0.0     GnarE  0.000000   
5736        ViegoQ  3.496903    ViegoW   0.0    ViegoE  0.000000   
5737  RyzeQWrapper  3.496903     RyzeW   0.0     RyzeE  0.000000   
5738        XayahQ  0.000000    XayahW   0.0    XayahE  0.258713   
5739       ThreshQ  6.174561   ThreshW   0.0   ThreshE  0.000000   

            r_name  r_cd  d_name        d_cd  f_name        f_cd  \
0           AkaliR   0.0       4    0.965248       8    0.965248   
1         EvelynnR   0.0       4    0.965248       9    0.965248   
2         CamilleR   0.0       4    0.965248       8    0.965248   
3          EzrealR   0.0       6    0.965248       4    0.965248   
4           RakanR   0.0       2    0.965248       4    0.965248   
...            ...   ...     ...         ...     ...         ...   
5735         GnarR   0.0       8  321.221800       4  244.177440   
5736        ViegoR   0.0       9   13.755020       4  184.033580   
5737         RyzeR   0.0       8    0.000000       4    0.000000   
5738        XayahR   0.0       4    0.000000       3    0.000000   
5739  ThreshRPenta   0.0       4    0.000000       2    0.000000   

      distance_from_player_x  distance_from_player_z  
0                    140.000                194.0000  
1                    106.000                244.0000  
2                     22.000                 34.0000  
3                      0.000                  0.0000  
4                    472.000                146.0000  
...                      ...                     ...  
5735                7283.955              12402.9683  
5736                1243.068               5825.7573  
5737                2194.932               7219.7573  
5738                 682.258                404.5527  
5739                 678.802                511.3061  

[5740 rows x 14 columns]

### Convert Minion and Turret Names into Index

In [15]:
MINION_NAMES   = list(df_s["minions"].name.unique())
TURRET_NAMES   = list(df_s["turrets"].name.unique())
MONSTER_NAMES  = list(df_s["monsters"].name.unique())

df_s["minions"]["name"] = df_s["minions"]["name"].apply(lambda d: MINION_NAMES.index(d))
df_s["turrets"]["name"] = df_s["turrets"]["name"].apply(lambda d: TURRET_NAMES.index(d))
df_s["monsters"]["name"] = df_s["monsters"]["name"].apply(lambda d: MONSTER_NAMES.index(d))

In [16]:
df_s["missiles"].head()

time   missile_name        spell_name  start_pos_x  start_pos_z  \
0  40.420643         XayahE  XayahBasicAttack    10881.035     3304.213   
1  40.698338         XayahE  XayahBasicAttack    10881.035     3304.213   
2  84.601630  eyeofthestorm             RyzeE     7783.628     8220.531   
3  84.874176  eyeofthestorm             RyzeE     7783.628     8220.531   
4  85.158110  eyeofthestorm             RyzeE     7783.628     8220.531   

   end_pos_x  end_pos_z       pos_x     pos_z  
0  10332.000  3112.0000  10334.0000  3114.000  
1  10332.000  3112.0000  10334.0000  3114.000  
2   7554.915  7600.0103   7625.2993  7730.157  
3   7554.915  7600.0103   7554.1055  7478.331  
4   7554.915  7600.0103   7554.1055  7478.331

### Data Cleaning for all Game Object Dataframes

In [19]:
for obj in GAME_OBJECT_LIST:
    if obj != "missiles":
        df_s[obj] = df_s[obj].drop_duplicates(
            subset=["time", "name"])
    else:
        df_s[obj] = df_s[obj].drop_duplicates(
            subset=["time", "missile_name"])
    df_s[obj] = df_s[obj][
        df_s[obj]["time"] > 15]

### Data Normalisation for Champs DF

In [20]:
df_s["champs"].loc[df_s["champs"]['q_cd'] < 0, 'q_cd'] = 0
df_s["champs"].loc[df_s["champs"]['w_cd'] < 0, 'w_cd'] = 0
df_s["champs"].loc[df_s["champs"]['e_cd'] < 0, 'e_cd'] = 0
df_s["champs"].loc[df_s["champs"]['r_cd'] < 0, 'r_cd'] = 0
df_s["champs"].loc[df_s["champs"]['d_cd'] < 0, 'd_cd'] = 0
df_s["champs"].loc[df_s["champs"]['f_cd'] < 0, 'f_cd'] = 0

## Observation Spec

### Init `replay_df`

In [26]:
replay_df = pd.DataFrame()
times = df_s["champs"].drop_duplicates(subset=['time'])["time"]
replay_df["time"] = times

### Add Distance Between Local Player and All Game Objects

In [28]:
player_df = df_s["champs"][df_s["champs"]["name"] == CHAMP_LIST.index("Ezreal")]

In [29]:
def add_distances(original_df):
    # Step 1: Filter out Player's data
    player_df_data = player_df[['time', 'pos_x', 'pos_z']]

    # Step 2: Merge with the original DataFrame on 'time'
    merged_df = original_df.merge(player_df_data, on='time', suffixes=('', '_player'))

    # Step 3: Calculate Euclidean distance
    merged_df['distance_from_player_x'] = abs(
        merged_df["pos_x_player"] - merged_df["pos_x"])
    merged_df['distance_from_player_z'] = abs(
        merged_df["pos_z_player"] - merged_df["pos_z"])
    merged_df['distance_from_player'] = np.sqrt(
        (merged_df['pos_x'] - merged_df['pos_x_player'])**2 +
        (merged_df['pos_z'] - merged_df['pos_z_player'])**2)
    merged_df = merged_df.drop(columns=["pos_x_player", "pos_z_player"])

    # Filter out the rows where the champion is Ezreal, as we don't need the distance of Ezreal to himself
    return merged_df

for obj in GAME_OBJECT_LIST:
    df_s[obj] = add_distances(df_s[obj])

### Flatten Observations

In [30]:
testdf_s = {}

#### Flatten Each Dataframe

In [31]:
times = df_s["champs"]["time"].unique()

def flatten_obs(df, max_objs, times):
    # print(df.columns)
    new_cols = [f"{c}_{idx}"
                for idx in range(max_objs)
                for c in df.columns[1:]]
    new_cols = ["time"] + new_cols
    obs   = []
    for tm in times:
        cur = df[df["time"] == tm]
        vals = cur.values[:, 1:]
        vals = vals[:max_objs, :]

        if cur.shape[0] < max_objs:
            padding_val = max_objs - vals.shape[0]
            padding = np.zeros((padding_val, vals.shape[1]))
            vals = np.vstack((vals, padding))
    
        # Flatten
        new_vals = np.hstack(vals)

        # Append
        obs.append(new_vals)

    # Combine
    obs   = np.vstack(obs)
    times = np.expand_dims(np.array(times), axis=1)
    obs   = np.hstack((times, obs))
    # print(obs.shape)
    obs_df = pd.DataFrame(data=obs, columns=new_cols)

    return obs_df

for obj, max_objs in zip(GAME_OBJECT_LIST, MAX_OBJS):
    # print("OBJ, MAX:", obj, max_objs)
    testdf_s[obj] = flatten_obs(df_s[obj], max_objs, times)

#### Combine All Flattened Dataframes

In [32]:
times_unsqueeze = np.expand_dims(times, 1)
replay_df_vals = [df.iloc[:, 1:] for df in testdf_s.values()]
replay_df_vals = np.hstack(replay_df_vals)
print(times_unsqueeze.shape, replay_df_vals.shape)
replay_df_vals = np.hstack((times_unsqueeze, replay_df_vals))

(574, 1) (574, 2250)


In [33]:
replay_df_vals.shape

(574, 2251)

In [34]:
def flatten_list(lst):
    return [x for xs in lst for x in xs]

replay_df_cols = [list(testdf_s[k].columns[1:].values) for k in testdf_s.keys()]
replay_df_cols = flatten_list(replay_df_cols)
replay_df_cols = ["time"] + replay_df_cols

In [36]:
replay_df = pd.DataFrame(
    data=replay_df_vals,
    columns=replay_df_cols)
replay_df

time name_0      hp_0 max_hp_0     mana_0 max_mana_0    armor_0  \
0    15.034752      2     570.0    570.0      200.0      200.0       35.0   
1    15.303469      2     570.0    570.0      200.0      200.0       35.0   
2    15.581064      2     570.0    570.0      200.0      200.0       35.0   
3    15.849633      2     570.0    570.0      200.0      200.0       35.0   
4     16.13916      2     570.0    570.0      200.0      200.0       35.0   
..         ...    ...       ...      ...        ...        ...        ...   
569  178.67474      2  574.6728   771.68  78.277725      300.0  38.384003   
570  178.97098      2  576.5816   771.68  84.027725      300.0  38.384003   
571  179.31879      2  576.5816   771.68  84.027725      300.0  38.384003   
572  179.63614      2  577.9464   771.68  89.777725      300.0  38.384003   
573  179.91972      2  579.3112   771.68  95.527725      300.0  38.384003   

       mr_0    ad_0 ap_0  ... ap_29 level_29 atk_range_29 visible_29 team_29  \
0      37.0    67.4  0.0  ...   0.0      0.0          0.0        0.0     0.0   
1      37.0    67.4  0.0  ...   0.0      0.0          0.0        0.0     0.0   
2      37.0    67.4  0.0  ...   0.0      0.0          0.0        0.0     0.0   
3      37.0    67.4  0.0  ...   0.0      0.0          0.0        0.0     0.0   
4      37.0    67.4  0.0  ...   0.0      0.0          0.0        0.0     0.0   
..      ...     ...  ...  ...   ...      ...          ...        ...     ...   
569  38.476  69.776  0.0  ...   0.0      0.0          0.0        0.0     0.0   
570  38.476  69.776  0.0  ...   0.0      0.0          0.0        0.0     0.0   
571  38.476  69.776  0.0  ...   0.0      0.0          0.0        0.0     0.0   
572  38.476  69.776  0.0  ...   0.0      0.0          0.0        0.0     0.0   
573  38.476  69.776  0.0  ...   0.0      0.0          0.0        0.0     0.0   

    pos_x_29 pos_z_29 distance_from_player_x_29 distance_from_player_z_29  \
0        0.0      0.0                       0.0                       0.0   
1        0.0      0.0                       0.0                       0.0   
2        0.0      0.0                       0.0                       0.0   
3        0.0      0.0                       0.0                       0.0   
4        0.0      0.0                       0.0                       0.0   
..       ...      ...                       ...                       ...   
569      0.0      0.0                       0.0                       0.0   
570      0.0      0.0                       0.0                       0.0   
571      0.0      0.0                       0.0                       0.0   
572      0.0      0.0                       0.0                       0.0   
573      0.0      0.0                       0.0                       0.0   

    distance_from_player_29  
0                       0.0  
1                       0.0  
2                       0.0  
3                       0.0  
4                       0.0  
..                      ...  
569                     0.0  
570                     0.0  
571                     0.0  
572                     0.0  
573                     0.0  

[574 rows x 2251 columns]

## Action Spec

In [37]:
player_df

time  name      hp  max_hp       mana  max_mana    armor       mr  \
503    15.034752    30  700.00  700.00  375.00000    375.00  36.0000  30.0000   
513    15.303469    30  700.00  700.00  375.00000    375.00  36.0000  30.0000   
523    15.581064    30  700.00  700.00  375.00000    375.00  36.0000  30.0000   
533    15.849633    30  700.00  700.00  375.00000    375.00  36.0000  30.0000   
543    16.139160    30  700.00  700.00  375.00000    375.00  36.0000  30.0000   
...          ...   ...     ...     ...        ...       ...      ...      ...   
6643  178.674740    30  850.45  850.45  327.76978    478.25  42.9325  31.9175   
6653  178.970980    30  850.45  850.45  328.76727    478.25  42.9325  31.9175   
6663  179.318790    30  850.45  850.45  328.76727    478.25  42.9325  31.9175   
6683  179.636140    30  850.45  850.45  329.76477    478.25  42.9325  31.9175   
6693  179.919720    30  850.45  850.45  330.76227    478.25  42.9325  31.9175   

           ad   ap  ...   w_name  w_cd   e_name       e_cd   r_name  r_cd  \
503   77.4000  0.0  ...  EzrealW   0.0  EzrealE   0.000000  EzrealR   0.0   
513   77.4000  0.0  ...  EzrealW   0.0  EzrealE   0.000000  EzrealR   0.0   
523   77.4000  0.0  ...  EzrealW   0.0  EzrealE   0.000000  EzrealR   0.0   
533   77.4000  0.0  ...  EzrealW   0.0  EzrealE   0.000000  EzrealR   0.0   
543   77.4000  0.0  ...  EzrealW   0.0  EzrealE   0.000000  EzrealR   0.0   
...       ...  ...  ...      ...   ...      ...        ...      ...   ...   
6643  81.0875  0.0  ...  EzrealW   0.0  EzrealE  13.740845  EzrealR   0.0   
6653  81.0875  0.0  ...  EzrealW   0.0  EzrealE  13.444611  EzrealR   0.0   
6663  81.0875  0.0  ...  EzrealW   0.0  EzrealE  13.096802  EzrealR   0.0   
6683  81.0875  0.0  ...  EzrealW   0.0  EzrealE  12.779449  EzrealR   0.0   
6693  81.0875  0.0  ...  EzrealW   0.0  EzrealE  12.495865  EzrealR   0.0   

     d_name      d_cd f_name      f_cd  
503       6  0.965248      4  0.965248  
513       6  0.696531      4  0.696531  
523       6  0.418936      4  0.418936  
533       6  0.150367      4  0.150367  
543       6  0.000000      4  0.000000  
...     ...       ...    ...       ...  
6643      6  0.000000      4  0.000000  
6653      6  0.000000      4  0.000000  
6663      6  0.000000      4  0.000000  
6683      6  0.000000      4  0.000000  
6693      6  0.000000      4  0.000000  

[574 rows x 28 columns]

### Movement

In [38]:
import math

player_cur_pos  = player_df[["time", "pos_x", "pos_z"]]
player_next_pos = player_df[["time", "pos_x", "pos_z"]].shift(1)
player_next_pos = player_next_pos.fillna(0)
player_x_delta  = player_next_pos["pos_x"] - player_cur_pos["pos_x"]
player_z_delta  = player_next_pos["pos_z"] - player_cur_pos["pos_z"]
player_x_delta.iloc[0] = 0
player_z_delta.iloc[0] = 0

player_x_delta_digit = (player_x_delta / 100).round().clip(-4, +4)
player_z_delta_digit = (player_z_delta / 100).round().clip(-4, +4)

player_df["player_x_delta"]       = player_x_delta
player_df["player_z_delta"]       = player_z_delta
player_df["player_delta"]         = np.sqrt(player_x_delta ** 2 + player_z_delta ** 2)
player_df["player_x_delta_digit"] = player_x_delta_digit
player_df["player_z_delta_digit"] = player_z_delta_digit

/var/folders/zb/zb7f652n559gx4c68jjjxzfh0000gn/T/ipykernel_3723/524912525.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df["player_x_delta"]       = player_x_delta
/var/folders/zb/zb7f652n559gx4c68jjjxzfh0000gn/T/ipykernel_3723/524912525.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df["player_z_delta"]       = player_z_delta
/var/folders/zb/zb7f652n559gx4c68jjjxzfh0000gn/T/ipykernel_3723/524912525.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [39]:
player_movement_df_final = \
    player_df[["time", "player_x_delta_digit", "player_z_delta_digit"]]
player_movement_df_final

time  player_x_delta_digit  player_z_delta_digit
503    15.034752                   0.0                   0.0
513    15.303469                   1.0                   0.0
523    15.581064                   1.0                  -0.0
533    15.849633                  -0.0                  -1.0
543    16.139160                  -1.0                  -1.0
...          ...                   ...                   ...
6643  178.674740                  -0.0                  -0.0
6653  178.970980                   0.0                   0.0
6663  179.318790                  -0.0                   1.0
6683  179.636140                  -1.0                  -1.0
6693  179.919720                  -1.0                  -1.0

[574 rows x 3 columns]

In [40]:
replay_df = pd.merge(replay_df, player_movement_df_final, on="time")
replay_df

time name_0      hp_0 max_hp_0     mana_0 max_mana_0    armor_0  \
0    15.034752      2     570.0    570.0      200.0      200.0       35.0   
1    15.303469      2     570.0    570.0      200.0      200.0       35.0   
2    15.581064      2     570.0    570.0      200.0      200.0       35.0   
3    15.849633      2     570.0    570.0      200.0      200.0       35.0   
4     16.13916      2     570.0    570.0      200.0      200.0       35.0   
..         ...    ...       ...      ...        ...        ...        ...   
569  178.67474      2  574.6728   771.68  78.277725      300.0  38.384003   
570  178.97098      2  576.5816   771.68  84.027725      300.0  38.384003   
571  179.31879      2  576.5816   771.68  84.027725      300.0  38.384003   
572  179.63614      2  577.9464   771.68  89.777725      300.0  38.384003   
573  179.91972      2  579.3112   771.68  95.527725      300.0  38.384003   

       mr_0    ad_0 ap_0  ... atk_range_29 visible_29 team_29 pos_x_29  \
0      37.0    67.4  0.0  ...          0.0        0.0     0.0      0.0   
1      37.0    67.4  0.0  ...          0.0        0.0     0.0      0.0   
2      37.0    67.4  0.0  ...          0.0        0.0     0.0      0.0   
3      37.0    67.4  0.0  ...          0.0        0.0     0.0      0.0   
4      37.0    67.4  0.0  ...          0.0        0.0     0.0      0.0   
..      ...     ...  ...  ...          ...        ...     ...      ...   
569  38.476  69.776  0.0  ...          0.0        0.0     0.0      0.0   
570  38.476  69.776  0.0  ...          0.0        0.0     0.0      0.0   
571  38.476  69.776  0.0  ...          0.0        0.0     0.0      0.0   
572  38.476  69.776  0.0  ...          0.0        0.0     0.0      0.0   
573  38.476  69.776  0.0  ...          0.0        0.0     0.0      0.0   

    pos_z_29 distance_from_player_x_29 distance_from_player_z_29  \
0        0.0                       0.0                       0.0   
1        0.0                       0.0                       0.0   
2        0.0                       0.0                       0.0   
3        0.0                       0.0                       0.0   
4        0.0                       0.0                       0.0   
..       ...                       ...                       ...   
569      0.0                       0.0                       0.0   
570      0.0                       0.0                       0.0   
571      0.0                       0.0                       0.0   
572      0.0                       0.0                       0.0   
573      0.0                       0.0                       0.0   

    distance_from_player_29 player_x_delta_digit player_z_delta_digit  
0                       0.0                  0.0                  0.0  
1                       0.0                  1.0                  0.0  
2                       0.0                  1.0                 -0.0  
3                       0.0                 -0.0                 -1.0  
4                       0.0                 -1.0                 -1.0  
..                      ...                  ...                  ...  
569                     0.0                 -0.0                 -0.0  
570                     0.0                  0.0                  0.0  
571                     0.0                 -0.0                  1.0  
572                     0.0                 -1.0                 -1.0  
573                     0.0                 -1.0                 -1.0  

[574 rows x 2253 columns]

### Recall (Not Using this Yet)

In [41]:
still_frames_matches = ((player_df["player_x_delta_digit"] == 0) & \
(player_df["player_z_delta_digit"] == 0))
still_frames_idx = still_frames_matches[still_frames_matches].index
still_frames = player_df.loc[still_frames_idx]

still_frames["time_diff"] = still_frames["time"].diff().fillna(0)
still_frames['consecutive'] = still_frames['time_diff'] < 1

still_frames['group'] = (~still_frames['consecutive']).cumsum()

# Get the start and end times for each group
group_start_end_times = still_frames.groupby('group')['time'].agg(['min', 'max'])

# Calculate the duration of each group
group_start_end_times['duration'] = group_start_end_times['max'] - group_start_end_times['min']

# Find groups where the duration is at least 8 seconds
long_groups = group_start_end_times[group_start_end_times['duration'] >= 8].index

# Extract all rows belonging to these groups
long_duration_groups = still_frames[still_frames['group'].isin(long_groups)]
long_duration_groups

Empty DataFrame
Columns: [time, name, hp, max_hp, mana, max_mana, armor, mr, ad, ap, level, atk_range, visible, team, pos_x, pos_z, q_name, q_cd, w_name, w_cd, e_name, e_cd, r_name, r_cd, d_name, d_cd, f_name, f_cd, player_x_delta, player_z_delta, player_delta, player_x_delta_digit, player_z_delta_digit, time_diff, consecutive, group]
Index: []

[0 rows x 36 columns]

### Spells

In [42]:
def swap_columns(df, col1, col2):
    """
    Swap two columns in a pandas DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame in which to swap columns.
    col1 (str): The name of the first column to swap.
    col2 (str): The name of the second column to swap.
    """
    temp = df[col1].copy()
    df[col1] = df[col2]
    df[col2] = temp
    return df

In [43]:
d_name = player_df["d_name"]
if d_name.iloc[0] != SUMMONER_NAMES.index("SummonerFlash"):
    swap_columns(player_df, "d_name", "f_name")
    swap_columns(player_df, "d_cd", "f_cd")

/var/folders/zb/zb7f652n559gx4c68jjjxzfh0000gn/T/ipykernel_3723/3959099287.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col1] = df[col2]
/var/folders/zb/zb7f652n559gx4c68jjjxzfh0000gn/T/ipykernel_3723/3959099287.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col2] = temp


In [44]:
d_name = player_df["d_name"]
d_name

503     4
513     4
523     4
533     4
543     4
       ..
6643    4
6653    4
6663    4
6683    4
6693    4
Name: d_name, Length: 574, dtype: int64

In [45]:
spell_casts = player_df[["time", "q_cd", "w_cd", "e_cd", "r_cd", "d_cd", "f_cd"]]

#### Q

In [54]:
# Get every time q was cast (this will be 100% acc)
qcd_s_cur  = spell_casts["q_cd"]
qcd_s_prev = spell_casts["q_cd"].shift(+1).fillna(0)
qcd_s_diff = qcd_s_cur - qcd_s_prev
q_cast     = (qcd_s_diff > 4) & (qcd_s_cur > 0)
# spell_casts_full = spell_casts.merge(q_cast, on=["time"])
print("Number of ezreal q casts:", q_cast.sum())

# Look through ezreal missiles to find the ezreal q's
# (This will be less than 100% acc because of timing issue with scraping
# missiles)
ezreal_q_df = pd.read_sql(
    "SELECT time, start_pos_x, start_pos_z, end_pos_x, end_pos_z FROM missiles WHERE spell_name = 'EzrealQ';",
    con)
ezreal_q_df = ezreal_q_df.drop_duplicates(
    subset=['start_pos_x', 'start_pos_z'])
print(ezreal_q_df)

# Get q spell aim as digital
ezreal_q_start_pos  = ezreal_q_df[["time", "start_pos_x", "start_pos_z"]]
ezreal_q_end_pos = ezreal_q_df[["time", "end_pos_x", "end_pos_z"]]
ezreal_q_end_pos = ezreal_q_end_pos.fillna(0)
ezreal_q_x_delta  = ezreal_q_end_pos["end_pos_x"] - ezreal_q_start_pos["start_pos_x"]
ezreal_q_z_delta  = ezreal_q_end_pos["end_pos_z"] - ezreal_q_start_pos["start_pos_z"]

ezreal_q_x_delta_digit = (ezreal_q_x_delta / 100).round().clip(-4, +4)
ezreal_q_z_delta_digit = (ezreal_q_z_delta / 100).round().clip(-4, +4)

ezreal_q_df["ezreal_q_x_delta"] = ezreal_q_x_delta
ezreal_q_df["ezreal_q_z_delta"] = ezreal_q_z_delta
ezreal_q_df["ezreal_q_x_delta_digit"] = ezreal_q_x_delta_digit
ezreal_q_df["ezreal_q_z_delta_digit"] = ezreal_q_z_delta_digit

Number of ezreal q casts: 9
          time  start_pos_x  start_pos_z  end_pos_x  end_pos_z
0   117.788864    11437.131    1661.6350  12563.790  2071.8050
2   127.895790    10879.960    1182.8914  11011.325  2374.6733
5   138.633450    11398.467    1294.7552  12315.749  2066.8910
8   152.625460    10730.425    1034.5089  11340.131  2066.9136
10  172.179610    10034.534    1048.1306  10985.169  1778.8120
15  178.416630    10251.595     886.4006  11126.689  1706.0410


In [55]:
ezreal_q_df_final = \
    ezreal_q_df[["time", "ezreal_q_x_delta_digit", "ezreal_q_z_delta_digit"]]
ezreal_q_df_final["using_q"] = 1
ezreal_q_df_final

/var/folders/zb/zb7f652n559gx4c68jjjxzfh0000gn/T/ipykernel_3723/1335434122.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ezreal_q_df_final["using_q"] = 1


time  ezreal_q_x_delta_digit  ezreal_q_z_delta_digit  using_q
0   117.788864                     4.0                     4.0        1
2   127.895790                     1.0                     4.0        1
5   138.633450                     4.0                     4.0        1
8   152.625460                     4.0                     4.0        1
10  172.179610                     4.0                     4.0        1
15  178.416630                     4.0                     4.0        1

In [56]:
replay_df = pd.merge(replay_df, ezreal_q_df_final, on="time", how="left")

In [57]:
replay_df["using_q"] = replay_df["using_q"].fillna(0)
replay_df = replay_df.fillna(0)
replay_df

time  name_0      hp_0  max_hp_0      mana_0  max_mana_0  \
0     15.034752       2  570.0000    570.00  200.000000       200.0   
1     15.303469       2  570.0000    570.00  200.000000       200.0   
2     15.581064       2  570.0000    570.00  200.000000       200.0   
3     15.849633       2  570.0000    570.00  200.000000       200.0   
4     16.139160       2  570.0000    570.00  200.000000       200.0   
..          ...     ...       ...       ...         ...         ...   
569  178.674740       2  574.6728    771.68   78.277725       300.0   
570  178.970980       2  576.5816    771.68   84.027725       300.0   
571  179.318790       2  576.5816    771.68   84.027725       300.0   
572  179.636140       2  577.9464    771.68   89.777725       300.0   
573  179.919720       2  579.3112    771.68   95.527725       300.0   

       armor_0    mr_0    ad_0  ap_0  ...  player_z_delta_digit  \
0    35.000000  37.000  67.400   0.0  ...                   0.0   
1    35.000000  37.000  67.400   0.0  ...                   0.0   
2    35.000000  37.000  67.400   0.0  ...                  -0.0   
3    35.000000  37.000  67.400   0.0  ...                  -1.0   
4    35.000000  37.000  67.400   0.0  ...                  -1.0   
..         ...     ...     ...   ...  ...                   ...   
569  38.384003  38.476  69.776   0.0  ...                  -0.0   
570  38.384003  38.476  69.776   0.0  ...                   0.0   
571  38.384003  38.476  69.776   0.0  ...                   1.0   
572  38.384003  38.476  69.776   0.0  ...                  -1.0   
573  38.384003  38.476  69.776   0.0  ...                  -1.0   

     ezreal_q_x_delta_digit_x  ezreal_q_z_delta_digit_x  using_q_x  \
0                         0.0                       0.0          0   
1                         0.0                       0.0          0   
2                         0.0                       0.0          0   
3                         0.0                       0.0          0   
4                         0.0                       0.0          0   
..                        ...                       ...        ...   
569                       0.0                       0.0          0   
570                       0.0                       0.0          0   
571                       0.0                       0.0          0   
572                       0.0                       0.0          0   
573                       0.0                       0.0          0   

     ezreal_q_x_delta_digit_y  ezreal_q_z_delta_digit_y using_q_y  \
0                         0.0                       0.0       0.0   
1                         0.0                       0.0       0.0   
2                         0.0                       0.0       0.0   
3                         0.0                       0.0       0.0   
4                         0.0                       0.0       0.0   
..                        ...                       ...       ...   
569                       0.0                       0.0       0.0   
570                       0.0                       0.0       0.0   
571                       0.0                       0.0       0.0   
572                       0.0                       0.0       0.0   
573                       0.0                       0.0       0.0   

     ezreal_q_x_delta_digit ezreal_q_z_delta_digit  using_q  
0                       0.0                    0.0      0.0  
1                       0.0                    0.0      0.0  
2                       0.0                    0.0      0.0  
3                       0.0                    0.0      0.0  
4                       0.0                    0.0      0.0  
..                      ...                    ...      ...  
569                     0.0                    0.0      0.0  
570                     0.0                    0.0      0.0  
571                     0.0                    0.0      0.0  
572                     0.0                    0.0      0.0  
5

#### W (not verified)

In [62]:
# Get every time w was cast (this will be 100% acc)
wcd_s_cur  = spell_casts["w_cd"]
wcd_s_prev = spell_casts["w_cd"].shift(+1).fillna(0)
wcd_s_diff = wcd_s_cur - wcd_s_prev
w_cast     = (wcd_s_diff > 11) & (wcd_s_cur > 0)
# spell_casts_full = spell_casts.merge(w_cast, on=["time"])
print("Number of ezreal w casts:", w_cast.sum())

# Look through ezreal missiles to find the ezreal w's
# (This will be less than 100% acc because of timing issue with scraping
# missiles)
ezreal_w_df = pd.read_sql(
    "SELECT time, start_pos_x, start_pos_z, end_pos_x, end_pos_z FROM missiles WHERE spell_name = 'EzrealW';",
    con)
ezreal_w_df = ezreal_w_df.drop_duplicates(
    subset=['start_pos_x', 'start_pos_z'])
#ezreal_w_df

# Get w spell aim as digital
ezreal_w_start_pos  = ezreal_w_df[["time", "start_pos_x", "start_pos_z"]]
ezreal_w_end_pos = ezreal_w_df[["time", "end_pos_x", "end_pos_z"]]
ezreal_w_end_pos = ezreal_w_end_pos.fillna(0)
ezreal_w_x_delta  = ezreal_w_end_pos["end_pos_x"] - ezreal_w_start_pos["start_pos_x"]
ezreal_w_z_delta  = ezreal_w_end_pos["end_pos_z"] - ezreal_w_start_pos["start_pos_z"]

ezreal_w_x_delta_digit = (ezreal_w_x_delta / 100).round().clip(-4, +4)
ezreal_w_z_delta_digit = (ezreal_w_z_delta / 100).round().clip(-4, +4)

ezreal_w_df["ezreal_w_x_delta"] = ezreal_w_x_delta
ezreal_w_df["ezreal_w_z_delta"] = ezreal_w_z_delta
ezreal_w_df["ezreal_w_x_delta_digit"] = ezreal_w_x_delta_digit
ezreal_w_df["ezreal_w_z_delta_digit"] = ezreal_w_z_delta_digit

Number of ezreal w casts: 0


In [63]:
ezreal_w_df_final = \
    ezreal_w_df[["time", "ezreal_w_x_delta_digit", "ezreal_w_z_delta_digit"]]
ezreal_w_df_final["using_w"] = 1

In [64]:
replay_df = pd.merge(replay_df, ezreal_w_df_final, on="time", how="left")

In [65]:
replay_df

time  name_0      hp_0  max_hp_0      mana_0  max_mana_0  \
0     15.034752       2  570.0000    570.00  200.000000       200.0   
1     15.303469       2  570.0000    570.00  200.000000       200.0   
2     15.581064       2  570.0000    570.00  200.000000       200.0   
3     15.849633       2  570.0000    570.00  200.000000       200.0   
4     16.139160       2  570.0000    570.00  200.000000       200.0   
..          ...     ...       ...       ...         ...         ...   
569  178.674740       2  574.6728    771.68   78.277725       300.0   
570  178.970980       2  576.5816    771.68   84.027725       300.0   
571  179.318790       2  576.5816    771.68   84.027725       300.0   
572  179.636140       2  577.9464    771.68   89.777725       300.0   
573  179.919720       2  579.3112    771.68   95.527725       300.0   

       armor_0    mr_0    ad_0  ap_0  ...  using_q_x  \
0    35.000000  37.000  67.400   0.0  ...          0   
1    35.000000  37.000  67.400   0.0  ...          0   
2    35.000000  37.000  67.400   0.0  ...          0   
3    35.000000  37.000  67.400   0.0  ...          0   
4    35.000000  37.000  67.400   0.0  ...          0   
..         ...     ...     ...   ...  ...        ...   
569  38.384003  38.476  69.776   0.0  ...          0   
570  38.384003  38.476  69.776   0.0  ...          0   
571  38.384003  38.476  69.776   0.0  ...          0   
572  38.384003  38.476  69.776   0.0  ...          0   
573  38.384003  38.476  69.776   0.0  ...          0   

     ezreal_q_x_delta_digit_y  ezreal_q_z_delta_digit_y  using_q_y  \
0                         0.0                       0.0        0.0   
1                         0.0                       0.0        0.0   
2                         0.0                       0.0        0.0   
3                         0.0                       0.0        0.0   
4                         0.0                       0.0        0.0   
..                        ...                       ...        ...   
569                       0.0                       0.0        0.0   
570                       0.0                       0.0        0.0   
571                       0.0                       0.0        0.0   
572                       0.0                       0.0        0.0   
573                       0.0                       0.0        0.0   

     ezreal_q_x_delta_digit  ezreal_q_z_delta_digit using_q  \
0                       0.0                     0.0     0.0   
1                       0.0                     0.0     0.0   
2                       0.0                     0.0     0.0   
3                       0.0                     0.0     0.0   
4                       0.0                     0.0     0.0   
..                      ...                     ...     ...   
569                     0.0                     0.0     0.0   
570                     0.0                     0.0     0.0   
571                     0.0                     0.0     0.0   
572                     0.0                     0.0     0.0   
573                     0.0                     0.0     0.0   

     ezreal_w_x_delta_digit ezreal_w_z_delta_digit  using_w  
0                       NaN                    NaN      NaN  
1                       NaN                    NaN      NaN  
2                       NaN                    NaN      NaN  
3                       NaN                    NaN      NaN  
4                       NaN                    NaN      NaN  
..                      ...                    ...      ...  
569                     NaN                    NaN      NaN  
570                     NaN                    NaN      NaN  
571                     NaN                    NaN      NaN  
572                     NaN                    NaN      NaN  
573                     NaN                    NaN      NaN  

[574 rows x 2265 columns]

In [66]:
replay_df["using_w"] = replay_df["using_w"].fillna(0)
replay_df = replay_df.fillna(0)
replay_df

time  name_0      hp_0  max_hp_0      mana_0  max_mana_0  \
0     15.034752       2  570.0000    570.00  200.000000       200.0   
1     15.303469       2  570.0000    570.00  200.000000       200.0   
2     15.581064       2  570.0000    570.00  200.000000       200.0   
3     15.849633       2  570.0000    570.00  200.000000       200.0   
4     16.139160       2  570.0000    570.00  200.000000       200.0   
..          ...     ...       ...       ...         ...         ...   
569  178.674740       2  574.6728    771.68   78.277725       300.0   
570  178.970980       2  576.5816    771.68   84.027725       300.0   
571  179.318790       2  576.5816    771.68   84.027725       300.0   
572  179.636140       2  577.9464    771.68   89.777725       300.0   
573  179.919720       2  579.3112    771.68   95.527725       300.0   

       armor_0    mr_0    ad_0  ap_0  ...  using_q_x  \
0    35.000000  37.000  67.400   0.0  ...          0   
1    35.000000  37.000  67.400   0.0  ...          0   
2    35.000000  37.000  67.400   0.0  ...          0   
3    35.000000  37.000  67.400   0.0  ...          0   
4    35.000000  37.000  67.400   0.0  ...          0   
..         ...     ...     ...   ...  ...        ...   
569  38.384003  38.476  69.776   0.0  ...          0   
570  38.384003  38.476  69.776   0.0  ...          0   
571  38.384003  38.476  69.776   0.0  ...          0   
572  38.384003  38.476  69.776   0.0  ...          0   
573  38.384003  38.476  69.776   0.0  ...          0   

     ezreal_q_x_delta_digit_y  ezreal_q_z_delta_digit_y  using_q_y  \
0                         0.0                       0.0        0.0   
1                         0.0                       0.0        0.0   
2                         0.0                       0.0        0.0   
3                         0.0                       0.0        0.0   
4                         0.0                       0.0        0.0   
..                        ...                       ...        ...   
569                       0.0                       0.0        0.0   
570                       0.0                       0.0        0.0   
571                       0.0                       0.0        0.0   
572                       0.0                       0.0        0.0   
573                       0.0                       0.0        0.0   

     ezreal_q_x_delta_digit  ezreal_q_z_delta_digit using_q  \
0                       0.0                     0.0     0.0   
1                       0.0                     0.0     0.0   
2                       0.0                     0.0     0.0   
3                       0.0                     0.0     0.0   
4                       0.0                     0.0     0.0   
..                      ...                     ...     ...   
569                     0.0                     0.0     0.0   
570                     0.0                     0.0     0.0   
571                     0.0                     0.0     0.0   
572                     0.0                     0.0     0.0   
573                     0.0                     0.0     0.0   

     ezreal_w_x_delta_digit ezreal_w_z_delta_digit  using_w  
0                         0                      0      0.0  
1                         0                      0      0.0  
2                         0                      0      0.0  
3                         0                      0      0.0  
4                         0                      0      0.0  
..                      ...                    ...      ...  
569                       0                      0      0.0  
570                       0                      0      0.0  
571                       0                      0      0.0  
572                       0                      0      0.0  
573                       0                      0      0.0  

[574 rows x 2265 columns]

In [67]:
replay_df

time  name_0      hp_0  max_hp_0      mana_0  max_mana_0  \
0     15.034752       2  570.0000    570.00  200.000000       200.0   
1     15.303469       2  570.0000    570.00  200.000000       200.0   
2     15.581064       2  570.0000    570.00  200.000000       200.0   
3     15.849633       2  570.0000    570.00  200.000000       200.0   
4     16.139160       2  570.0000    570.00  200.000000       200.0   
..          ...     ...       ...       ...         ...         ...   
569  178.674740       2  574.6728    771.68   78.277725       300.0   
570  178.970980       2  576.5816    771.68   84.027725       300.0   
571  179.318790       2  576.5816    771.68   84.027725       300.0   
572  179.636140       2  577.9464    771.68   89.777725       300.0   
573  179.919720       2  579.3112    771.68   95.527725       300.0   

       armor_0    mr_0    ad_0  ap_0  ...  using_q_x  \
0    35.000000  37.000  67.400   0.0  ...          0   
1    35.000000  37.000  67.400   0.0  ...          0   
2    35.000000  37.000  67.400   0.0  ...          0   
3    35.000000  37.000  67.400   0.0  ...          0   
4    35.000000  37.000  67.400   0.0  ...          0   
..         ...     ...     ...   ...  ...        ...   
569  38.384003  38.476  69.776   0.0  ...          0   
570  38.384003  38.476  69.776   0.0  ...          0   
571  38.384003  38.476  69.776   0.0  ...          0   
572  38.384003  38.476  69.776   0.0  ...          0   
573  38.384003  38.476  69.776   0.0  ...          0   

     ezreal_q_x_delta_digit_y  ezreal_q_z_delta_digit_y  using_q_y  \
0                         0.0                       0.0        0.0   
1                         0.0                       0.0        0.0   
2                         0.0                       0.0        0.0   
3                         0.0                       0.0        0.0   
4                         0.0                       0.0        0.0   
..                        ...                       ...        ...   
569                       0.0                       0.0        0.0   
570                       0.0                       0.0        0.0   
571                       0.0                       0.0        0.0   
572                       0.0                       0.0        0.0   
573                       0.0                       0.0        0.0   

     ezreal_q_x_delta_digit  ezreal_q_z_delta_digit using_q  \
0                       0.0                     0.0     0.0   
1                       0.0                     0.0     0.0   
2                       0.0                     0.0     0.0   
3                       0.0                     0.0     0.0   
4                       0.0                     0.0     0.0   
..                      ...                     ...     ...   
569                     0.0                     0.0     0.0   
570                     0.0                     0.0     0.0   
571                     0.0                     0.0     0.0   
572                     0.0                     0.0     0.0   
573                     0.0                     0.0     0.0   

     ezreal_w_x_delta_digit ezreal_w_z_delta_digit  using_w  
0                         0                      0      0.0  
1                         0                      0      0.0  
2                         0                      0      0.0  
3                         0                      0      0.0  
4                         0                      0      0.0  
..                      ...                    ...      ...  
569                       0                      0      0.0  
570                       0                      0      0.0  
571                       0                      0      0.0  
572                       0                      0      0.0  
573                       0                      0      0.0  

[574 rows x 2265 columns]

#### E

In [68]:
# Get every time e was cast (this will be 100% acc)
ecd_s_cur  = spell_casts["e_cd"]
ecd_s_prev = spell_casts["e_cd"].shift(+1).fillna(0)
ecd_s_diff = ecd_s_cur - ecd_s_prev
e_cast     = (ecd_s_diff > 25) & (ecd_s_cur > 0)
print("Number of ezreal e casts:", e_cast.sum())
e_cast_idx_s = e_cast[e_cast].index

# create a dataframe with all of e cast action params
e_cast_vals = []
e_cast_cols = ["time", "ezreal_e_x_delta_digit", "ezreal_e_z_delta_digit"]
for e_cast_idx in e_cast_idx_s:
    # e_cast_idx = e_cast_idx_s[0]
    e_row = player_df.loc[e_cast_idx-50:e_cast_idx+50][["time", "player_delta"]]
    e_row_idx = e_row.idxmax()["player_delta"]
    tm, x, y = player_df.loc[e_row_idx][["time", "player_x_delta_digit", "player_z_delta_digit"]]
    e_cast_vals.append([tm, x, y])

ezreal_e_df_final = pd.DataFrame(data=e_cast_vals, columns=e_cast_cols)
ezreal_e_df_final["using_e"] = 1
ezreal_e_df_final["using_e"] = ezreal_e_df_final["using_e"].fillna(0)
ezreal_e_df_final = ezreal_e_df_final.fillna(0)
ezreal_e_df_final

Number of ezreal e casts: 1


time  ezreal_e_x_delta_digit  ezreal_e_z_delta_digit  using_e
0  168.84871                     4.0                     4.0        1

In [69]:
replay_df = pd.merge(replay_df, ezreal_e_df_final, on="time", how="left")

In [70]:
replay_df

time  name_0      hp_0  max_hp_0      mana_0  max_mana_0  \
0     15.034752       2  570.0000    570.00  200.000000       200.0   
1     15.303469       2  570.0000    570.00  200.000000       200.0   
2     15.581064       2  570.0000    570.00  200.000000       200.0   
3     15.849633       2  570.0000    570.00  200.000000       200.0   
4     16.139160       2  570.0000    570.00  200.000000       200.0   
..          ...     ...       ...       ...         ...         ...   
569  178.674740       2  574.6728    771.68   78.277725       300.0   
570  178.970980       2  576.5816    771.68   84.027725       300.0   
571  179.318790       2  576.5816    771.68   84.027725       300.0   
572  179.636140       2  577.9464    771.68   89.777725       300.0   
573  179.919720       2  579.3112    771.68   95.527725       300.0   

       armor_0    mr_0    ad_0  ap_0  ...  using_q_y  ezreal_q_x_delta_digit  \
0    35.000000  37.000  67.400   0.0  ...        0.0                     0.0   
1    35.000000  37.000  67.400   0.0  ...        0.0                     0.0   
2    35.000000  37.000  67.400   0.0  ...        0.0                     0.0   
3    35.000000  37.000  67.400   0.0  ...        0.0                     0.0   
4    35.000000  37.000  67.400   0.0  ...        0.0                     0.0   
..         ...     ...     ...   ...  ...        ...                     ...   
569  38.384003  38.476  69.776   0.0  ...        0.0                     0.0   
570  38.384003  38.476  69.776   0.0  ...        0.0                     0.0   
571  38.384003  38.476  69.776   0.0  ...        0.0                     0.0   
572  38.384003  38.476  69.776   0.0  ...        0.0                     0.0   
573  38.384003  38.476  69.776   0.0  ...        0.0                     0.0   

     ezreal_q_z_delta_digit  using_q  ezreal_w_x_delta_digit  \
0                       0.0      0.0                       0   
1                       0.0      0.0                       0   
2                       0.0      0.0                       0   
3                       0.0      0.0                       0   
4                       0.0      0.0                       0   
..                      ...      ...                     ...   
569                     0.0      0.0                       0   
570                     0.0      0.0                       0   
571                     0.0      0.0                       0   
572                     0.0      0.0                       0   
573                     0.0      0.0                       0   

     ezreal_w_z_delta_digit using_w  ezreal_e_x_delta_digit  \
0                         0     0.0                     NaN   
1                         0     0.0                     NaN   
2                         0     0.0                     NaN   
3                         0     0.0                     NaN   
4                         0     0.0                     NaN   
..                      ...     ...                     ...   
569                       0     0.0                     NaN   
570                       0     0.0                     NaN   
571                       0     0.0                     NaN   
572                       0     0.0                     NaN   
573                       0     0.0                     NaN   

    ezreal_e_z_delta_digit  using_e  
0                      NaN      NaN  
1                      NaN      NaN  
2                      NaN      NaN  
3                      NaN      NaN  
4                      NaN      NaN  
..                     ...      ...  
569                    NaN      NaN  
570                    NaN      NaN  
571                    NaN      NaN  
572                    NaN      NaN  
573                    NaN      NaN  

[574 rows x 2268 columns]

In [71]:
replay_df["using_e"] = replay_df["using_e"].fillna(0)
replay_df = replay_df.fillna(0)
replay_df

time  name_0      hp_0  max_hp_0      mana_0  max_mana_0  \
0     15.034752       2  570.0000    570.00  200.000000       200.0   
1     15.303469       2  570.0000    570.00  200.000000       200.0   
2     15.581064       2  570.0000    570.00  200.000000       200.0   
3     15.849633       2  570.0000    570.00  200.000000       200.0   
4     16.139160       2  570.0000    570.00  200.000000       200.0   
..          ...     ...       ...       ...         ...         ...   
569  178.674740       2  574.6728    771.68   78.277725       300.0   
570  178.970980       2  576.5816    771.68   84.027725       300.0   
571  179.318790       2  576.5816    771.68   84.027725       300.0   
572  179.636140       2  577.9464    771.68   89.777725       300.0   
573  179.919720       2  579.3112    771.68   95.527725       300.0   

       armor_0    mr_0    ad_0  ap_0  ...  using_q_y  ezreal_q_x_delta_digit  \
0    35.000000  37.000  67.400   0.0  ...        0.0                     0.0   
1    35.000000  37.000  67.400   0.0  ...        0.0                     0.0   
2    35.000000  37.000  67.400   0.0  ...        0.0                     0.0   
3    35.000000  37.000  67.400   0.0  ...        0.0                     0.0   
4    35.000000  37.000  67.400   0.0  ...        0.0                     0.0   
..         ...     ...     ...   ...  ...        ...                     ...   
569  38.384003  38.476  69.776   0.0  ...        0.0                     0.0   
570  38.384003  38.476  69.776   0.0  ...        0.0                     0.0   
571  38.384003  38.476  69.776   0.0  ...        0.0                     0.0   
572  38.384003  38.476  69.776   0.0  ...        0.0                     0.0   
573  38.384003  38.476  69.776   0.0  ...        0.0                     0.0   

     ezreal_q_z_delta_digit  using_q  ezreal_w_x_delta_digit  \
0                       0.0      0.0                       0   
1                       0.0      0.0                       0   
2                       0.0      0.0                       0   
3                       0.0      0.0                       0   
4                       0.0      0.0                       0   
..                      ...      ...                     ...   
569                     0.0      0.0                       0   
570                     0.0      0.0                       0   
571                     0.0      0.0                       0   
572                     0.0      0.0                       0   
573                     0.0      0.0                       0   

     ezreal_w_z_delta_digit using_w  ezreal_e_x_delta_digit  \
0                         0     0.0                     0.0   
1                         0     0.0                     0.0   
2                         0     0.0                     0.0   
3                         0     0.0                     0.0   
4                         0     0.0                     0.0   
..                      ...     ...                     ...   
569                       0     0.0                     0.0   
570                       0     0.0                     0.0   
571                       0     0.0                     0.0   
572                       0     0.0                     0.0   
573                       0     0.0                     0.0   

    ezreal_e_z_delta_digit  using_e  
0                      0.0      0.0  
1                      0.0      0.0  
2                      0.0      0.0  
3                      0.0      0.0  
4                      0.0      0.0  
..                     ...      ...  
569                    0.0      0.0  
570                    0.0      0.0  
571                    0.0      0.0  
572                    0.0      0.0  
573                    0.0      0.0  

[574 rows x 2268 columns]

#### Flash

In [73]:
# Get every time e was cast (this will be 100% acc)
dcd_s_cur  = spell_casts["d_cd"]
dcd_s_prev = spell_casts["d_cd"].shift(+1).fillna(0)
dcd_s_diff = dcd_s_cur - dcd_s_prev
d_cast     = (dcd_s_diff > 250) & (dcd_s_cur > 0)
d_cast_idx_s = d_cast[d_cast].index

d_cast_vals = []
d_cast_cols = ["time", "ezreal_d_x_delta_digit", "ezreal_d_z_delta_digit"]
for d_cast_idx in d_cast_idx_s:
    d_row = player_df.loc[d_cast_idx-50:d_cast_idx+50][["time", "player_delta"]]
    d_row_idx = d_row.idxmax()["player_delta"]
    tm, x, y = player_df.loc[d_row_idx][["time", "player_x_delta_digit", "player_z_delta_digit"]]
    d_cast_vals.append([tm, x, y])

ezreal_d_df_final = pd.DataFrame(data=d_cast_vals, columns=d_cast_cols)
ezreal_d_df_final["using_d"] = 1
ezreal_d_df_final

Empty DataFrame
Columns: [time, ezreal_d_x_delta_digit, ezreal_d_z_delta_digit, using_d]
Index: []

In [74]:
replay_df = pd.merge(replay_df, ezreal_d_df_final, on="time", how="left")

In [75]:
replay_df

time  name_0      hp_0  max_hp_0      mana_0  max_mana_0  \
0     15.034752       2  570.0000    570.00  200.000000       200.0   
1     15.303469       2  570.0000    570.00  200.000000       200.0   
2     15.581064       2  570.0000    570.00  200.000000       200.0   
3     15.849633       2  570.0000    570.00  200.000000       200.0   
4     16.139160       2  570.0000    570.00  200.000000       200.0   
..          ...     ...       ...       ...         ...         ...   
569  178.674740       2  574.6728    771.68   78.277725       300.0   
570  178.970980       2  576.5816    771.68   84.027725       300.0   
571  179.318790       2  576.5816    771.68   84.027725       300.0   
572  179.636140       2  577.9464    771.68   89.777725       300.0   
573  179.919720       2  579.3112    771.68   95.527725       300.0   

       armor_0    mr_0    ad_0  ap_0  ...  using_q  ezreal_w_x_delta_digit  \
0    35.000000  37.000  67.400   0.0  ...      0.0                       0   
1    35.000000  37.000  67.400   0.0  ...      0.0                       0   
2    35.000000  37.000  67.400   0.0  ...      0.0                       0   
3    35.000000  37.000  67.400   0.0  ...      0.0                       0   
4    35.000000  37.000  67.400   0.0  ...      0.0                       0   
..         ...     ...     ...   ...  ...      ...                     ...   
569  38.384003  38.476  69.776   0.0  ...      0.0                       0   
570  38.384003  38.476  69.776   0.0  ...      0.0                       0   
571  38.384003  38.476  69.776   0.0  ...      0.0                       0   
572  38.384003  38.476  69.776   0.0  ...      0.0                       0   
573  38.384003  38.476  69.776   0.0  ...      0.0                       0   

     ezreal_w_z_delta_digit  using_w  ezreal_e_x_delta_digit  \
0                         0      0.0                     0.0   
1                         0      0.0                     0.0   
2                         0      0.0                     0.0   
3                         0      0.0                     0.0   
4                         0      0.0                     0.0   
..                      ...      ...                     ...   
569                       0      0.0                     0.0   
570                       0      0.0                     0.0   
571                       0      0.0                     0.0   
572                       0      0.0                     0.0   
573                       0      0.0                     0.0   

     ezreal_e_z_delta_digit using_e  ezreal_d_x_delta_digit  \
0                       0.0     0.0                     NaN   
1                       0.0     0.0                     NaN   
2                       0.0     0.0                     NaN   
3                       0.0     0.0                     NaN   
4                       0.0     0.0                     NaN   
..                      ...     ...                     ...   
569                     0.0     0.0                     NaN   
570                     0.0     0.0                     NaN   
571                     0.0     0.0                     NaN   
572                     0.0     0.0                     NaN   
573                     0.0     0.0                     NaN   

    ezreal_d_z_delta_digit  using_d  
0                      NaN      NaN  
1                      NaN      NaN  
2                      NaN      NaN  
3                      NaN      NaN  
4                      NaN      NaN  
..                     ...      ...  
569                    NaN      NaN  
570                    NaN      NaN  
571                    NaN      NaN  
572                    NaN      NaN  
573                    NaN      NaN  

[574 rows x 2271 columns]

In [76]:
replay_df["using_d"] = replay_df["using_d"].fillna(0)
replay_df = replay_df.fillna(0)
replay_df

time  name_0      hp_0  max_hp_0      mana_0  max_mana_0  \
0     15.034752       2  570.0000    570.00  200.000000       200.0   
1     15.303469       2  570.0000    570.00  200.000000       200.0   
2     15.581064       2  570.0000    570.00  200.000000       200.0   
3     15.849633       2  570.0000    570.00  200.000000       200.0   
4     16.139160       2  570.0000    570.00  200.000000       200.0   
..          ...     ...       ...       ...         ...         ...   
569  178.674740       2  574.6728    771.68   78.277725       300.0   
570  178.970980       2  576.5816    771.68   84.027725       300.0   
571  179.318790       2  576.5816    771.68   84.027725       300.0   
572  179.636140       2  577.9464    771.68   89.777725       300.0   
573  179.919720       2  579.3112    771.68   95.527725       300.0   

       armor_0    mr_0    ad_0  ap_0  ...  using_q  ezreal_w_x_delta_digit  \
0    35.000000  37.000  67.400   0.0  ...      0.0                       0   
1    35.000000  37.000  67.400   0.0  ...      0.0                       0   
2    35.000000  37.000  67.400   0.0  ...      0.0                       0   
3    35.000000  37.000  67.400   0.0  ...      0.0                       0   
4    35.000000  37.000  67.400   0.0  ...      0.0                       0   
..         ...     ...     ...   ...  ...      ...                     ...   
569  38.384003  38.476  69.776   0.0  ...      0.0                       0   
570  38.384003  38.476  69.776   0.0  ...      0.0                       0   
571  38.384003  38.476  69.776   0.0  ...      0.0                       0   
572  38.384003  38.476  69.776   0.0  ...      0.0                       0   
573  38.384003  38.476  69.776   0.0  ...      0.0                       0   

     ezreal_w_z_delta_digit  using_w  ezreal_e_x_delta_digit  \
0                         0      0.0                     0.0   
1                         0      0.0                     0.0   
2                         0      0.0                     0.0   
3                         0      0.0                     0.0   
4                         0      0.0                     0.0   
..                      ...      ...                     ...   
569                       0      0.0                     0.0   
570                       0      0.0                     0.0   
571                       0      0.0                     0.0   
572                       0      0.0                     0.0   
573                       0      0.0                     0.0   

     ezreal_e_z_delta_digit using_e  ezreal_d_x_delta_digit  \
0                       0.0     0.0                       0   
1                       0.0     0.0                       0   
2                       0.0     0.0                       0   
3                       0.0     0.0                       0   
4                       0.0     0.0                       0   
..                      ...     ...                     ...   
569                     0.0     0.0                       0   
570                     0.0     0.0                       0   
571                     0.0     0.0                       0   
572                     0.0     0.0                       0   
573                     0.0     0.0                       0   

    ezreal_d_z_delta_digit  using_d  
0                        0      0.0  
1                        0      0.0  
2                        0      0.0  
3                        0      0.0  
4                        0      0.0  
..                     ...      ...  
569                      0      0.0  
570                      0      0.0  
571                      0      0.0  
572                      0      0.0  
573                      0      0.0  

[574 rows x 2271 columns]

#### Alt (Not using this yet)

In [77]:
fcd_s_cur  = spell_casts["f_cd"]
fcd_s_prev = spell_casts["f_cd"].shift(+1).fillna(0)
fcd_s_diff = fcd_s_cur - fcd_s_prev
f_cast     = (fcd_s_diff > 180) & (fcd_s_cur > 0)
f_cast_idx_s = f_cast[f_cast].index
print("Alt sum casts:", f_cast.sum())

Alt sum casts: 0


#### Auto Attack

In [78]:
"""
Determine auto attack target:
1. [x] Get all auto attack missile end positions
2. [x] Get all game object positions
3. [x] Match missile end locations with game object positions (within a tolerance)
4. [ ] Get IDX of target type
"""

# 1. Get all auto attack missile end positions
auto_attacks_df = pd.read_sql("SELECT * FROM missiles WHERE spell_name LIKE 'EzrealBasicAttack%';", con)
auto_attacks_df = auto_attacks_df.drop_duplicates(subset=["start_pos_x", "start_pos_z"])
auto_attacks_df

# 2. Get all game object positions
champ_pos_df    = df_s["champs"].drop_duplicates(subset=["pos_x", "pos_z"])
turrets_pos_df  = df_s["turrets"].drop_duplicates(subset=["pos_x", "pos_z"])
monsters_pos_df = df_s["monsters"].drop_duplicates(subset=["pos_x", "pos_z"])
minions_pos_df  = df_s["minions"].drop_duplicates(subset=["pos_x", "pos_z"])

# 3. Match missile end locations with game object positions (within a tolerance)
def find_aa_target(row):
    target_x, target_z = row["pos_x"], row["pos_z"]
    # For each champ pos, check against missile positions
    for _, aa in auto_attacks_df.iterrows():
        end_pos_x, end_pos_z = aa["end_pos_x"], aa["end_pos_z"]
        if target_x == end_pos_x and target_z == end_pos_z:
            return True
    return False

champ_found_aa    = champ_pos_df.apply(lambda r: find_aa_target(r), axis=1)
turrets_found_aa  = turrets_pos_df.apply(lambda r: find_aa_target(r), axis=1)
monsters_found_aa = monsters_pos_df.apply(lambda r: find_aa_target(r), axis=1)
minions_found_aa  = minions_pos_df.apply(lambda r: find_aa_target(r), axis=1)

print(auto_attacks_df.shape)
champ_found_aa.sum(), turrets_found_aa.sum(), monsters_found_aa.sum(), minions_found_aa.sum()

(7, 13)


(0, 0, 0, 5)

In [81]:
AUTO_TARGET_TYPES = ["champ", "turret", "monster", "minion"]

champ_autos    = champ_pos_df.loc[champ_found_aa[champ_found_aa].index][["time", "pos_x", "pos_z"]]
turrets_autos  = turrets_pos_df.loc[turrets_found_aa[turrets_found_aa].index][["time", "pos_x", "pos_z"]]
monsters_autos = monsters_pos_df.loc[monsters_found_aa[monsters_found_aa].index][["time", "pos_x", "pos_z"]]
minions_autos  = minions_pos_df.loc[minions_found_aa[minions_found_aa].index][["time", "pos_x", "pos_z"]]

champ_autos["auto_type"]    = "champ"
turrets_autos["auto_type"]  = "turret"
monsters_autos["auto_type"] = "monster"
minions_autos["auto_type"]  = "minion"

all_autos_cols = ["time", "auto_digit_x", "auto_digit_z", "target_type"]
all_autos_vals = np.vstack((champ_autos, turrets_autos, monsters_autos, minions_autos))
all_autos_df   = pd.DataFrame(data=all_autos_vals, columns=all_autos_cols)
all_autos_df["target_type"]   = all_autos_df["target_type"].apply(lambda d: AUTO_TARGET_TYPES.index(d))
all_autos_df["using_auto"] = 1
all_autos_df

time auto_digit_x auto_digit_z  target_type  using_auto
0  152.92659    11548.402    1578.0912            3           1
1   162.4802     11307.99    1596.6964            3           1
2  165.97435    11085.911     1589.094            3           1
3   170.0425    10857.082    1177.6383            3           1
4  170.77783      10626.0       1222.0            3           1

In [82]:
replay_df = pd.merge(replay_df, all_autos_df, on="time", how="left")

In [83]:
replay_df

time  name_0      hp_0  max_hp_0      mana_0  max_mana_0    armor_0  \
0    15.034752       2  570.0000    570.00  200.000000       200.0  35.000000   
1    15.303469       2  570.0000    570.00  200.000000       200.0  35.000000   
2    15.581064       2  570.0000    570.00  200.000000       200.0  35.000000   
3    15.849633       2  570.0000    570.00  200.000000       200.0  35.000000   
4     16.13916       2  570.0000    570.00  200.000000       200.0  35.000000   
..         ...     ...       ...       ...         ...         ...        ...   
569  178.67474       2  574.6728    771.68   78.277725       300.0  38.384003   
570  178.97098       2  576.5816    771.68   84.027725       300.0  38.384003   
571  179.31879       2  576.5816    771.68   84.027725       300.0  38.384003   
572  179.63614       2  577.9464    771.68   89.777725       300.0  38.384003   
573  179.91972       2  579.3112    771.68   95.527725       300.0  38.384003   

       mr_0    ad_0  ap_0  ...  ezreal_e_x_delta_digit  \
0    37.000  67.400   0.0  ...                     0.0   
1    37.000  67.400   0.0  ...                     0.0   
2    37.000  67.400   0.0  ...                     0.0   
3    37.000  67.400   0.0  ...                     0.0   
4    37.000  67.400   0.0  ...                     0.0   
..      ...     ...   ...  ...                     ...   
569  38.476  69.776   0.0  ...                     0.0   
570  38.476  69.776   0.0  ...                     0.0   
571  38.476  69.776   0.0  ...                     0.0   
572  38.476  69.776   0.0  ...                     0.0   
573  38.476  69.776   0.0  ...                     0.0   

     ezreal_e_z_delta_digit  using_e  ezreal_d_x_delta_digit  \
0                       0.0      0.0                       0   
1                       0.0      0.0                       0   
2                       0.0      0.0                       0   
3                       0.0      0.0                       0   
4                       0.0      0.0                       0   
..                      ...      ...                     ...   
569                     0.0      0.0                       0   
570                     0.0      0.0                       0   
571                     0.0      0.0                       0   
572                     0.0      0.0                       0   
573                     0.0      0.0                       0   

     ezreal_d_z_delta_digit  using_d auto_digit_x  auto_digit_z target_type  \
0                         0      0.0          NaN           NaN         NaN   
1                         0      0.0          NaN           NaN         NaN   
2                         0      0.0          NaN           NaN         NaN   
3                         0      0.0          NaN           NaN         NaN   
4                         0      0.0          NaN           NaN         NaN   
..                      ...      ...          ...           ...         ...   
569                       0      0.0          NaN           NaN         NaN   
570                       0      0.0          NaN           NaN         NaN   
571                       0      0.0          NaN           NaN         NaN   
572                       0      0.0          NaN           NaN         NaN   
573                       0      0.0          NaN           NaN         NaN   

     using_auto  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4           NaN  
..          ...  
569         NaN  
570         NaN  
571         NaN  
572         NaN  
573         NaN  

[574 rows x 2275 columns]

In [84]:
replay_df["using_auto"] = replay_df["using_auto"].fillna(0)
replay_df = replay_df.fillna(0)
replay_df

time  name_0      hp_0  max_hp_0      mana_0  max_mana_0  \
0     15.034752       2  570.0000    570.00  200.000000       200.0   
1     15.303469       2  570.0000    570.00  200.000000       200.0   
2     15.581064       2  570.0000    570.00  200.000000       200.0   
3     15.849633       2  570.0000    570.00  200.000000       200.0   
4     16.139160       2  570.0000    570.00  200.000000       200.0   
..          ...     ...       ...       ...         ...         ...   
569  178.674740       2  574.6728    771.68   78.277725       300.0   
570  178.970980       2  576.5816    771.68   84.027725       300.0   
571  179.318790       2  576.5816    771.68   84.027725       300.0   
572  179.636140       2  577.9464    771.68   89.777725       300.0   
573  179.919720       2  579.3112    771.68   95.527725       300.0   

       armor_0    mr_0    ad_0  ap_0  ...  ezreal_e_x_delta_digit  \
0    35.000000  37.000  67.400   0.0  ...                     0.0   
1    35.000000  37.000  67.400   0.0  ...                     0.0   
2    35.000000  37.000  67.400   0.0  ...                     0.0   
3    35.000000  37.000  67.400   0.0  ...                     0.0   
4    35.000000  37.000  67.400   0.0  ...                     0.0   
..         ...     ...     ...   ...  ...                     ...   
569  38.384003  38.476  69.776   0.0  ...                     0.0   
570  38.384003  38.476  69.776   0.0  ...                     0.0   
571  38.384003  38.476  69.776   0.0  ...                     0.0   
572  38.384003  38.476  69.776   0.0  ...                     0.0   
573  38.384003  38.476  69.776   0.0  ...                     0.0   

     ezreal_e_z_delta_digit  using_e  ezreal_d_x_delta_digit  \
0                       0.0      0.0                       0   
1                       0.0      0.0                       0   
2                       0.0      0.0                       0   
3                       0.0      0.0                       0   
4                       0.0      0.0                       0   
..                      ...      ...                     ...   
569                     0.0      0.0                       0   
570                     0.0      0.0                       0   
571                     0.0      0.0                       0   
572                     0.0      0.0                       0   
573                     0.0      0.0                       0   

     ezreal_d_z_delta_digit  using_d auto_digit_x  auto_digit_z target_type  \
0                         0      0.0          0.0           0.0         0.0   
1                         0      0.0          0.0           0.0         0.0   
2                         0      0.0          0.0           0.0         0.0   
3                         0      0.0          0.0           0.0         0.0   
4                         0      0.0          0.0           0.0         0.0   
..                      ...      ...          ...           ...         ...   
569                       0      0.0          0.0           0.0         0.0   
570                       0      0.0          0.0           0.0         0.0   
571                       0      0.0          0.0           0.0         0.0   
572                       0      0.0          0.0           0.0         0.0   
573                       0      0.0          0.0           0.0         0.0   

     using_auto  
0           0.0  
1           0.0  
2           0.0  
3           0.0  
4           0.0  
..          ...  
569         0.0  
570         0.0  
571         0.0  
572         0.0  
573         0.0  

[574 rows x 2275 columns]

## Get GameID

In [223]:
game_id = pd.read_sql("SELECT game_id FROM games;", con)
game_id

game_id
0  2843393945